In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from IPython.display import clear_output

from keras.preprocessing.text import Tokenizer
from nltk.stem import WordNetLemmatizer
from nlp_id.lemmatizer import Lemmatizer
from collections import Counter
from pandas import read_csv
from pandas import DataFrame
import pickle, re, json, time
from string import punctuation
import numpy as np
from requests import get
from io import StringIO
import nltk, re, json, csv
nltk.download("words")
from nltk.corpus import words
from tqdm import tqdm
import re

[nltk_data] Downloading package words to /home/yelf/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
df = read_csv('./pubg_dataset_terjemah/pubg_dataset_terjemah.csv')
df.columns
df.head()

,text,text_translated,text_terjemahan
0,tencent saya bukan player profesional tapi kam...,"Tencent, I'm not a professional player, but we...","Tencent, saya bukan pemain profesional, tapi k..."
1,bug setelah update turun pesawat tidak kebagia...,"bug after the update, getting off the plane di...","bug setelah update, turun dari pesawat tidak d..."
2,keseluruhan game nya sudah bagus apalagi setel...,"Overall the game is good, especially after the...","Secara keseluruhan gamenya bagus, apalagi sete..."
3,pubg setelah update bnyak saja masalahnya sepe...,After updating PUBG there were lots of problem...,Setelah update PUBG banyak sekali permasalahan...
4,game bagus tapi masih banyak bug masih perlu d...,good game but there are still lots of bugs tha...,permainan bagus tapi masih banyak bug yang mas...


In [3]:
# REMOVE SPECIAL CHARACTERS & CASE FOLDING
def cleaning(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', str(text))              # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text)                   # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text)                          # menghapus RT
    text = re.sub(r"http\S+", '', text)                         # menghapus link
    text = re.sub(r'[0-9]+', '', text)                          # menghapus angka
    text = re.sub(r"[,.;@#?!&$]+\ *", " ", text)                # mengganti tanda baca dengan spasi
    text = re.sub(r'[^\w\s]', '', text)                         # menghapus karakter selain huruf dan angka
    text = text.replace('\n', ' ')                              # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', punctuation))   # menghapus semua tanda baca
    text = text.strip(' ')                                      # menghapus karakter spasi dari kiri dan kanan teks
    text = re.sub(r' +', ' ', text)                             # Remove double space
    return text.lower()

In [4]:
df['text_clean'] = df['text_terjemahan'].apply(cleaning)

In [13]:
# Counter words function declaration
def counter_word(texts):
    count = Counter()
    for text in texts.values:
        for word in text.split():
            word = word.lower()
            count[word] += 1
    return count

# Count unique words
counter = counter_word(df['text_clean'])
num_unique_words = len(counter)     # 20993
# counter.most_common(5)
print("\nUnique words : ", num_unique_words)

# Tokenizer word from text_lemmatize coloumn
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(df['text_clean'])

# Get Word Index from tokenizer
word_index = tokenizer.word_index

with open('./word_dict.json', mode='w') as file:
    json.dump(counter, file, indent=4)


Unique words :  10741


In [14]:
# LEMMATIZING
def lemmatizerWords(text):
    words = text.split()
    lemmatized_words = []
    lemmatizer  = Lemmatizer()
    lemmatized_words.append(lemmatizer.lemmatize(word.lower()) for word in words)
    return ' '.join(lemmatized_words[0])

new_word_dict = {}
%time
start_time = time.time()
for word, count in tqdm(counter.items()):
    text_lemmatized = lemmatizerWords(word)
    # clear_output(wait=True)
    # print(word)
    # print(text_lemmatized)
    new_word_dict[text_lemmatized] = count
    
end_time = time.time()
execution_time = (end_time - start_time) / 60
print(f"Execution time in minutes: {execution_time:.6f}")

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 4.77 μs


100%|██████████| 10741/10741 [1:28:06<00:00,  2.03it/s]

Execution time in seconds: 5286.106488


In [5]:
print("New Unique Words : ", len(new_word_dict))

New Unique Words :  6965


In [34]:
with open('./new_word_dict.json', mode='w') as file:
    json.dump(new_word_dict, file, indent=4)
    
unword_df = DataFrame(new_word_dict.items(), columns=["Word", "Count"])
unword_df
unword_df.to_csv('./new_word_dict.csv', index=False)

# English Dictionary Check

In [47]:
# CHECk is the word exist on english dictionary
def is_english_word(word):
    english_words = set(words.words())
    return word.lower() not in english_words

word_dict = {}
not_word_dict = {}

i, j = 0, 0
for word, count in new_word_dict.items():
    if is_english_word(word):
        i+=1
        clear_output(wait=True)
        print(i, word)
        print("Skip : ", j)
        not_word_dict[word] = count
    else:
        j+=1
        clear_output(wait=True)
        print(i, word)
        print("Skip : ", j)
        word_dict[word] = count
        
print("In Dict : ", i)
print("Not In  : ", j)

3128 cheeter
Skip :  4556
In Dict :  3128
Not In  :  4556


In [48]:
len(not_word_dict)

3128

# Indonesia Dictionary Check

In [6]:
with open('./new_word_dict.json', mode='r') as file:
    new_word_dict = json.load(file)
    
print("New Unique Words : ", len(new_word_dict))

New Unique Words :  6965


In [7]:
from kbbi import KBBI

word_dict = {}
not_word_dict = {}

def in_indonesia_word(kata):
    return KBBI(kata)

i, j = 0, 0
%time
start_time = time.time()
for word, count in tqdm(new_word_dict.items()):
    try:
        if in_indonesia_word(word):
            # print("=> Word Dictioonary")
            # i+=1
            # clear_output(wait=True)
            # print("In Word Dict     : ", i, word)
            # print("Not In Word Dict : ", j)
            word_dict[word] = count
    except:
        # print("=> Not Word Dictionary")
        # j+=1
        # clear_output(wait=True)
        # print("In Word Dict     : ", i)
        # print("Not In Word Dict : ", j, word)
        not_word_dict[word] = count

print("In Dict : ", i)
print("Not In  : ", j)

end_time = time.time()
execution_time = (end_time - start_time) / 60
print(f"Execution time in minutes: {execution_time:.6f}")

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 4.05 μs


100%|██████████| 6965/6965 [2:10:51<00:00,  1.13s/it]   

In Dict :  0
Not In  :  0
Execution time in minutes: 130.852832


In [8]:
print("New Unique Words : ", len(not_word_dict))

New Unique Words :  6879


In [9]:
with open('./not_word_dict.json', mode='w') as file:
    json.dump(not_word_dict, file, indent=4)
    
unword_df = DataFrame(not_word_dict.items(), columns=["Word", "Count"])
unword_df
unword_df.to_csv('./not_word_dict.csv', index=False)
    
unword_df = DataFrame(not_word_dict.items(), columns=["Word", "Count"])
unword_df
unword_df.to_csv('./not_word_dict.csv', index=False)

In [10]:
with open('./word_dict.json', mode='w') as file:
    json.dump(word_dict, file, indent=4)
    
unword_df = DataFrame(word_dict.items(), columns=["Word", "Count"])
unword_df
unword_df.to_csv('./word_dict.json.csv', index=False)

# CEK DISINI

In [6]:
import json
# with open('./not_word_dict.json', mode='r') as file:
#     not_word_dict = json.load(file)
# print("Jumlah Kata : ", len(not_word_dict), "kata")

with open('./resources/slangwords/slangwords_dict.json', mode='r') as file:
    slangwords_dict = json.load(file)
# slangwords_dict

with open('./resources/stopwords/stopwords_dict.json', mode='r') as file:
    stopwords_dict = json.load(file)

In [7]:
# Remove slangwords
remove_word = []
for word, count in not_word_dict.items():
    if word in slangwords_dict:
        remove_word.append(word)

for word in remove_word:
    del not_word_dict[word]
print("Jumlah kata : ", len(not_word_dict), "kata")

Jumlah kata :  6034 kata


In [8]:
# Remove stopwords
remove_word = []
for word, count in not_word_dict.items():
    if word in stopwords_dict:
        remove_word.append(word)

for word in remove_word:
    del not_word_dict[word]
print("Jumlah kata : ", len(not_word_dict), "kata")

Jumlah kata :  5671 kata


In [9]:
# Remove "nya"
def remove_nya(kata):
    return re.sub(r'nya$', '', kata)

new_slangwords_dict = {}
for word, count in not_word_dict.items():
    if "nya" in word:
        fixed_word = remove_nya(word)
        # print(word, "==>", fixed_word)
        new_slangwords_dict[word] = fixed_word

with open('./resources/slangwords/new_slangwords_dict.json', mode="w") as file:
    json.dump(new_slangwords_dict, file, indent=4)
    
remove_word = []
for word, count in not_word_dict.items():
    if word in new_slangwords_dict:
        remove_word.append(word)
        
for word in remove_word:
    del not_word_dict[word]
    
print("Jumlah kata : ", len(not_word_dict), "kata")

Jumlah kata :  5280 kata


In [40]:
with open('./resources/slangwords/new_slangwords_dict.json', "r") as file:
    new_slangwords_dict = json.load(file)
len(new_slangwords_dict)

392

In [10]:
# Remove "di"
def remove_di(kata):
    return re.sub(r'^di', '', kata)

# new_slangwords_dict = {}
for word, count in not_word_dict.items():
    if "di" in word:
        fixed_word = remove_di(word)
        # print(word, "==>", fixed_word)
        if word == fixed_word:
            continue
        else:
            new_slangwords_dict[word] = fixed_word

with open('./resources/slangwords/new_slangwords_dict.json', mode="r+") as file:
    json.dump(new_slangwords_dict, file, indent=4)

In [13]:
len(new_slangwords_dict)

497

In [43]:
new_slangwords_dict

{'nyaman': 'nyaman',
 'sinyal': 'sinyal',
 'nyata': 'nyata',
 'gbnya': 'gb',
 'processornya': 'processor',
 'servernya': 'server',
 'ponselnya': 'ponsel',
 'satusatunya': 'satusatu',
 'hentihentinya': 'hentihenti',
 'msnya': 'ms',
 'mempedulikannya': 'mempedulikan',
 'ratenya': 'rate',
 'micnya': 'mic',
 'senyap': 'senyap',
 'karyawannya': 'karyawan',
 'kerjasamanya': 'kerjasama',
 'mengupgradenya': 'mengupgrade',
 'mengupdatenya': 'mengupdate',
 'nyangkut': 'nyangkut',
 'captchanya': 'captcha',
 'settingnya': 'setting',
 'nyawa': 'nyawa',
 'replaynya': 'replay',
 'originalnya': 'original',
 'skillnya': 'skill',
 'mpsnya': 'mps',
 'gemsnya': 'gems',
 'framingnya': 'framing',
 'damagenya': 'damage',
 'matmakingnya': 'matmaking',
 'matchmakingnya': 'matchmaking',
 'reloadnya': 'reload',
 'hdnya': 'hd',
 'recoilnya': 'recoil',
 'framedropnya': 'framedrop',
 'fixnya': 'fix',
 'menginstalnya': 'menginstal',
 'tbtbnya': 'tbtb',
 'renderingnya': 'rendering',
 'puzzlenya': 'puzzle',
 'stapnya'

In [45]:
print("Jumlah kata : ", len(not_word_dict), "kata")

Jumlah kata :  5280 kata


In [14]:
with open('./not_word_dict.json', mode='w') as file:
    json.dump(not_word_dict, file, indent=4)
    
unword_df = DataFrame(not_word_dict.items(), columns=["Word", "Count"])
unword_df
unword_df.to_csv('./not_word_dict.csv', index=False)

In [3]:
with open('./not_word_dict.json', mode='r') as file:
    not_word_dict = json.load(file)